In [2]:
from matplotlib import pyplot as plt
import pandas as pd
import json
import subprocess
import os
from pathlib import Path

polylla_exec = "./build/Debug/Polylla.exe"
kernel_exec = "./generate_kernel.py"
invert_exec = "./invert_faces.py"
analytics_exec = "./analytics.py"

iter = [10,20,30,40,50]
iter = [20]
points = [100, 1000, 10000]
repeat = 10

def create_analytics_file(name):
    with open(name, "w") as f:
        f.write("""file_name, min_angle, max_angle, min_edge_ratio, max_edge_ratio, \
avg_edge_ratio, min_area_ratio_kernel_poly, \
max_area_ratio_kernel_poly, avg_area_ratio_kernel_poly, \
min_apr, max_apr, avg_apr\n""")

In [3]:
pikachu_dir = "./smooth_experiments/pikachu/"
pikachu_tri = pikachu_dir + "pikachu_triangle_inverted.off"

pikachu_output_dir = pikachu_dir + "results/"
pikachu_lap_dir = pikachu_output_dir + "lap/"
pikachu_con_lap_dir = pikachu_output_dir + "con_lap/"
pikachu_distmesh_dir = pikachu_output_dir + "distmesh/"
Path(pikachu_lap_dir).mkdir(parents=True, exist_ok=True)
Path(pikachu_con_lap_dir).mkdir(parents=True, exist_ok=True)
Path(pikachu_distmesh_dir).mkdir(parents=True, exist_ok=True)

analytics_pikachu = pikachu_output_dir + "analytics.csv"

In [8]:
subprocess.run([polylla_exec, pikachu_tri, pikachu_output_dir + "pikachu_vanilla"], stdout=subprocess.DEVNULL)

# laplacian
for n in iter:
    print("n:",n)
    for i in range(repeat):
        print("i:",i,end=" ")
        subprocess.run([polylla_exec, pikachu_tri, pikachu_lap_dir + "pikachu_" + str(n) + "lap-"+str(i), "--smooth", "laplacian", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
    print("")

# constrained laplacian
for n in iter:
    print("n:",n)
    for i in range(repeat):
        print("i:",i,end=" ")
        subprocess.run([polylla_exec, pikachu_tri, pikachu_con_lap_dir + "pikachu_" + str(n) + "con_lap-"+str(i), "--smooth", "laplacian-edge-ratio", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
    print("")

# distmesh
for n in iter:
    print("n:",n)
    for i in range(repeat):
        print("i:",i,end=" ")
        subprocess.run([polylla_exec, pikachu_tri, pikachu_distmesh_dir + "pikachu_" + str(n) + "distmesh-"+str(i), "--smooth", "distmesh", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
    print("")

n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 


In [71]:
create_analytics_file(analytics_pikachu)

subprocess.run(["python.exe", kernel_exec, pikachu_output_dir + "pikachu_vanilla.off"], stdout=subprocess.DEVNULL)
subprocess.run(["python.exe", analytics_exec, pikachu_output_dir + "pikachu_vanilla.off", analytics_pikachu], stdout=subprocess.DEVNULL)

# laplacian
for n in iter:
    subprocess.run(["python.exe", kernel_exec, pikachu_lap_dir + "pikachu_" + str(n) + "lap-0.off"], stdout=subprocess.DEVNULL)
    subprocess.run(["python.exe", analytics_exec, pikachu_lap_dir + "pikachu_" + str(n) + "lap-0.off", analytics_pikachu], stdout=subprocess.DEVNULL)
    
# constrained laplacian
for n in iter:
    subprocess.run(["python.exe", kernel_exec, pikachu_con_lap_dir + "pikachu_" + str(n) + "con_lap-0.off"], stdout=subprocess.DEVNULL)
    subprocess.run(["python.exe", analytics_exec, pikachu_con_lap_dir + "pikachu_" + str(n) + "con_lap-0.off", analytics_pikachu], stdout=subprocess.DEVNULL)

# distmesh
for n in iter:
    subprocess.run(["python.exe", kernel_exec, pikachu_distmesh_dir + "pikachu_" + str(n) + "distmesh-0.off"], stdout=subprocess.DEVNULL)
    subprocess.run(["python.exe", analytics_exec, pikachu_distmesh_dir + "pikachu_" + str(n) + "distmesh-0.off", analytics_pikachu], stdout=subprocess.DEVNULL)


In [12]:
total_time = 0
smooth_time = 0
avg_smooth_time = 0
remaining_time = 0
for i in range(repeat):
    with open(pikachu_lap_dir + f"pikachu_20lap-{i}.json") as f:
        data = json.load(f)
        n = data["n_smooth_iterations"]
        smooth_time += data["time_to_smooth"]
        avg_smooth_time += data["time_to_smooth"]/n
        total_time += data["time_to_generate_polygonal_mesh"]
        remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]
print("Laplacian")
print("Total:",total_time/repeat)
print("Smooth", smooth_time/repeat)
print("Avg:", avg_smooth_time/repeat)
print("Remaining:", remaining_time/repeat)
print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))

total_time = 0
smooth_time = 0
avg_smooth_time = 0
remaining_time = 0
for i in range(repeat):
    with open(pikachu_con_lap_dir + f"pikachu_20con_lap-{i}.json") as f:
        data = json.load(f)
        n = data["n_smooth_iterations"]
        smooth_time += data["time_to_smooth"]
        avg_smooth_time += data["time_to_smooth"]/20
        total_time += data["time_to_generate_polygonal_mesh"]
        remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

print("Constrained Laplacian")
print("Total:",total_time/repeat)
print("Smooth", smooth_time/repeat)
print("Avg:", avg_smooth_time/repeat)
print("Remaining:", remaining_time/repeat)
print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))

total_time = 0
smooth_time = 0
avg_smooth_time = 0
remaining_time = 0
for i in range(repeat):
    with open(pikachu_distmesh_dir + f"pikachu_20lap-{i}.json") as f:
        data = json.load(f)
        n = data["n_smooth_iterations"]
        smooth_time += data["time_to_smooth"]
        avg_smooth_time += data["time_to_smooth"]/n
        total_time += data["time_to_generate_polygonal_mesh"]
        remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

print("Distmesh")
print("Total:",total_time/repeat)
print("Smooth", smooth_time/repeat)
print("Avg:", avg_smooth_time/repeat)
print("Remaining:", remaining_time/repeat)
print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))

Laplacian
Total: 2.3
Smooth 1.46177
Avg: 0.0730885
Remaining: 0.8382299999999999
One iter: 0.08020082989646321
Constrained Laplacian
Total: 45.85352
Smooth 45.13090999999999
Avg: 2.2565454999999996
Remaining: 0.7226100000000016
One iter: 0.7574446852472114
Distmesh
Total: 2.4702699999999997
Smooth 1.7538900000000002
Avg: 0.08769450000000001
Remaining: 0.7163800000000001
One iter: 0.109062655263909


In [2]:
# region_dir = "./smooth_experiments/region/"
# region_tri = region_dir + "magallanes.1.off"

# region_output_dir = region_dir + "results/"
# region_lap_dir = region_output_dir + "lap/"
# region_con_lap_dir = region_output_dir + "con_lap/"
# region_distmesh_dir = region_output_dir + "distmesh/"
# Path(region_lap_dir).mkdir(parents=True, exist_ok=True)
# Path(region_con_lap_dir).mkdir(parents=True, exist_ok=True)
# Path(region_distmesh_dir).mkdir(parents=True, exist_ok=True)

# analytics_region = region_output_dir + "analytics.csv"

# subprocess.run([polylla_exec, region_tri, region_output_dir + "region_vanilla"], stdout=subprocess.DEVNULL)

# region_iter = [10, 20]

# print("laplacian")
# for n in region_iter:
#     print("n:", n)
#     for i in range(repeat):
#         print("i:",i,end="")
#         subprocess.run([polylla_exec, region_tri, region_lap_dir + f"region_{str(n)}lap-{str(i)}", "--smooth", "laplacian", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
#     print("")


# print("constrained laplacian")
# for n in region_iter:
#     print("n:", n)
#     for i in range(repeat):
#         print("i:",i,end="")
#         subprocess.run([polylla_exec, region_tri, region_con_lap_dir + f"region_{str(n)}con_lap-{str(i)}", "--smooth", "laplacian-edge-ratio", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
#     print("")


# print("distmesh")
# for n in region_iter:
#     print("n:", n)
#     for i in range(repeat):
#         print("i:",i,end="")
#         subprocess.run([polylla_exec, region_tri, region_distmesh_dir + f"region_{str(n)}distmesh-{str(i)}", "--smooth", "distmesh", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
#     print("")


laplacian
n: 10
i: 0
n: 20
i: 0
constrained laplacian
n: 10
i: 0
n: 20
i: 0
distmesh
n: 10
i: 0
n: 20
i: 0


In [3]:
# create_analytics_file(analytics_region)

# print("vanilla-kernel")
# subprocess.run(["python.exe", kernel_exec, region_output_dir + "region_vanilla.off"], stdout=subprocess.DEVNULL)
# print("vanilla-analytics")
# subprocess.run(["python.exe", analytics_exec, region_output_dir + f"region_{n}lap-0.off", analytics_region], stdout=subprocess.DEVNULL)

# for n in region_iter:
#     print("laplacian-kernel")
#     subprocess.run(["python.exe", kernel_exec, region_lap_dir + f"region_{str(n)}lap-0.off"], stdout=subprocess.DEVNULL)
#     print("constrained-laplacian-kernel")
#     subprocess.run(["python.exe", kernel_exec, region_con_lap_dir + f"region_{str(n)}con_lap-0.off"], stdout=subprocess.DEVNULL)
#     # print("distmesh-kernel")
#     # subprocess.run(["python.exe", kernel_exec, region_distmesh_dir + f"region_{str(n)}distmesh-0.off"], stdout=subprocess.DEVNULL)
#     print("laplacian-analytics")
#     subprocess.run(["python.exe", analytics_exec, region_lap_dir + f"region_{n}lap-0.off", analytics_region], stdout=subprocess.DEVNULL)
#     print("constrained-laplacian-analytics")
#     subprocess.run(["python.exe", analytics_exec, region_con_lap_dir + f"region_{n}lap-0.off", analytics_region], stdout=subprocess.DEVNULL)
#     # print("distmesh-analytics")
#     # subprocess.run(["python.exe", analytics_exec, region_distmesh_dir + f"region_{n}lap-0.off", analytics_region], stdout=subprocess.DEVNULL)

vanilla-kernel


In [9]:
points = [100, 1000, 10000]

for i in points:
    print(i, "points")
    semi_dir = f"./smooth_experiments/semiuniform/{i}/"
    semi_tri = semi_dir + f"{i}semiUniform.off"

    semi_output_dir = semi_dir + "results/"
    semi_lap_dir = semi_output_dir + "lap/"
    semi_con_lap_dir = semi_output_dir + "con_lap/"
    semi_distmesh_dir = semi_output_dir + "distmesh/"
    Path(semi_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(semi_con_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(semi_distmesh_dir).mkdir(parents=True, exist_ok=True)

    subprocess.run([polylla_exec, semi_tri, semi_output_dir + "semi_vanilla"], stdout=subprocess.DEVNULL)

    # laplacian
    print("laplacian")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, semi_tri, semi_lap_dir + f"semi_{n}lap-{i}", "--smooth", "laplacian", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

    # constrained laplacian
    print("constrained laplacian")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, semi_tri, semi_con_lap_dir + f"semi_{n}con_lap-{i}", "--smooth", "laplacian-edge-ratio", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

    # distmesh
    print("distmesh")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, semi_tri, semi_distmesh_dir + f"semi_{n}distmesh-{i}", "--smooth", "distmesh", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

100 points
laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
1000 points
laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
10000 points
laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 


In [75]:

for i in points:
    print(i, "points")
    semi_dir = f"./smooth_experiments/semiuniform/{i}/"
    semi_tri = semi_dir + f"{i}semiUniform.off"

    semi_output_dir = semi_dir + "results/"
    semi_lap_dir = semi_output_dir + "lap/"
    semi_con_lap_dir = semi_output_dir + "con_lap/"
    semi_distmesh_dir = semi_output_dir + "distmesh/"
    Path(semi_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(semi_con_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(semi_distmesh_dir).mkdir(parents=True, exist_ok=True)

    analytics_semi = semi_output_dir + "analytics.csv"
    create_analytics_file(analytics_semi)

    subprocess.run(["python.exe", kernel_exec, semi_output_dir + "semi_vanilla.off"], stdout=subprocess.DEVNULL)
    subprocess.run(["python.exe", analytics_exec, semi_output_dir + "semi_vanilla.off", analytics_semi], stdout=subprocess.DEVNULL)

    # laplacian
    print("laplacian")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, semi_lap_dir + f"semi_{n}lap-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, semi_lap_dir + f"semi_{n}lap-0.off", analytics_semi], stdout=subprocess.DEVNULL)
    
    # constrained laplacian
    print("constrained laplacian")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, semi_con_lap_dir + f"semi_{n}con_lap-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, semi_con_lap_dir + f"semi_{n}con_lap-0.off", analytics_semi], stdout=subprocess.DEVNULL)

    # distmesh
    print("distmesh")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, semi_distmesh_dir + f"semi_{n}distmesh-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, semi_distmesh_dir + f"semi_{n}distmesh-0.off", analytics_semi], stdout=subprocess.DEVNULL)




100 points
laplacian
n: 10
n: 20
n: 30
n: 40
n: 50
constrained laplacian
n: 10
n: 20
n: 30
n: 40
n: 50
distmesh
n: 10
n: 20
n: 30
n: 40
n: 50
1000 points
laplacian
n: 10
n: 20
n: 30
n: 40
n: 50
constrained laplacian
n: 10
n: 20
n: 30
n: 40
n: 50
distmesh
n: 10
n: 20
n: 30
n: 40
n: 50
10000 points
laplacian
n: 10
n: 20
n: 30
n: 40
n: 50
constrained laplacian
n: 10
n: 20
n: 30
n: 40
n: 50
distmesh
n: 10
n: 20
n: 30
n: 40
n: 50


In [16]:
for p in points:
    semi_dir = f"./smooth_experiments/semiuniform/{p}/"
    semi_tri = semi_dir + f"{p}semiUniform.off"

    semi_output_dir = semi_dir + "results/"
    semi_lap_dir = semi_output_dir + "lap/"
    semi_con_lap_dir = semi_output_dir + "con_lap/"
    semi_distmesh_dir = semi_output_dir + "distmesh/"
    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(semi_lap_dir + f"semi_20lap-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/n
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]
    print("Laplacian", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))

for p in points:
    semi_dir = f"./smooth_experiments/semiuniform/{p}/"
    semi_tri = semi_dir + f"{p}semiUniform.off"

    semi_output_dir = semi_dir + "results/"
    semi_lap_dir = semi_output_dir + "lap/"
    semi_con_lap_dir = semi_output_dir + "con_lap/"
    semi_distmesh_dir = semi_output_dir + "distmesh/"
    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(semi_con_lap_dir + f"semi_20con_lap-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/20
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

    print("Constrained Laplacian", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))

for p in points:
    semi_dir = f"./smooth_experiments/semiuniform/{p}/"
    semi_tri = semi_dir + f"{p}semiUniform.off"

    semi_output_dir = semi_dir + "results/"
    semi_lap_dir = semi_output_dir + "lap/"
    semi_con_lap_dir = semi_output_dir + "con_lap/"
    semi_distmesh_dir = semi_output_dir + "distmesh/"
    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(semi_distmesh_dir + f"semi_20distmesh-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/n
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

    print("Distmesh", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))

Laplacian 100
Total: 0.9883
Smooth 0.67967
Avg: 0.0339835
Remaining: 0.30863
One iter: 0.0991890278695965
Laplacian 1000
Total: 9.97428
Smooth 7.438560000000001
Avg: 0.371928
Remaining: 2.5357200000000004
One iter: 0.1279136951928156
Laplacian 10000
Total: 111.31670000000001
Smooth 83.85272
Avg: 4.192636
Remaining: 27.46398
One iter: 0.13244106697949018
Constrained Laplacian 100
Total: 15.57865
Smooth 15.325479999999999
Avg: 0.7662740000000001
Remaining: 0.25317000000000006
One iter: 0.751658747317165
Constrained Laplacian 1000
Total: 220.8029
Smooth 218.24179999999996
Avg: 10.912090000000003
Remaining: 2.561100000000002
One iter: 0.809911386984077
Constrained Laplacian 10000
Total: 2350.9260000000004
Smooth 2323.127
Avg: 116.15635
Remaining: 27.799000000000113
One iter: 0.8068915118472492
Distmesh 100
Total: 20.691129999999998
Smooth 20.44077
Avg: 1.0220385
Remaining: 0.25036000000000025
One iter: 0.8032377435213888
Distmesh 1000
Total: 319.4941
Smooth 316.7016
Avg: 15.83508
Remaining

In [76]:
unif_dir = "./smooth_experiments/uniform/500/"
unif_inv_tri = unif_dir + "500uniform.off"
subprocess.run(["python.exe", invert_exec, unif_inv_tri], stdout=subprocess.DEVNULL)
unif_tri = unif_dir + "500uniform_inverted.off"

unif_output_dir = unif_dir + "results/"
Path(unif_output_dir).mkdir(parents=True, exist_ok=True)

analytics = unif_output_dir + "analytics.csv"
create_analytics_file(analytics)

subprocess.run([polylla_exec, unif_tri, unif_output_dir + "uniform_vanilla"], stdout=subprocess.DEVNULL)
subprocess.run(["python.exe", kernel_exec, unif_output_dir + "uniform_vanilla.off"], stdout=subprocess.DEVNULL)
subprocess.run([polylla_exec, unif_tri, unif_output_dir + "uniform_lap", "--smooth", "laplacian", "--smooth-iter", "1"], stdout=subprocess.DEVNULL)
subprocess.run(["python.exe", kernel_exec, unif_output_dir + "uniform_lap.off"], stdout=subprocess.DEVNULL)
subprocess.run([polylla_exec, unif_tri, unif_output_dir + "uniform_con_lap", "--smooth", "laplacian-edge-ratop", "--smooth-iter", "1"], stdout=subprocess.DEVNULL)
subprocess.run(["python.exe", kernel_exec, unif_output_dir + "uniform_con_lap.off"], stdout=subprocess.DEVNULL)
subprocess.run([polylla_exec, unif_tri, unif_output_dir + "uniform_distmesh", "--smooth", "distmesh", "--smooth-iter", "1"], stdout=subprocess.DEVNULL)
subprocess.run(["python.exe", kernel_exec, unif_output_dir + "uniform_distmesh.off"], stdout=subprocess.DEVNULL)

CompletedProcess(args=['python.exe', './generate_kernel.py', './smooth_experiments/uniform/500/results/uniform_distmesh.off'], returncode=0)

In [10]:
points = [100, 1000, 10000]
# iter = [10, 20]

for i in points:
    poisson_dir = f"./smooth_experiments/poisson/{i}/"
    poisson_tri = poisson_dir + f"{i}poisson.off"

    poisson_output_dir = poisson_dir + "results/"
    poisson_lap_dir = poisson_output_dir + "lap/"
    poisson_con_lap_dir = poisson_output_dir + "con_lap/"
    poisson_distmesh_dir = poisson_output_dir + "distmesh/"
    Path(poisson_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(poisson_con_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(poisson_distmesh_dir).mkdir(parents=True, exist_ok=True)

    subprocess.run([polylla_exec, poisson_tri, poisson_output_dir + "poisson_vanilla"], stdout=subprocess.DEVNULL)

    # laplacian
    print("laplacian")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, poisson_tri, poisson_lap_dir + f"poisson_{n}lap-{i}", "--smooth", "laplacian", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

    # constrained laplacian
    print("constrained laplacian")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, poisson_tri, poisson_con_lap_dir + f"poisson_{n}con_lap-{i}", "--smooth", "laplacian-edge-ratio", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

    # distmesh
    print("distmesh")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, poisson_tri, poisson_distmesh_dir + f"poisson_{n}distmesh-{i}", "--smooth", "distmesh", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")


laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 


In [78]:

for i in points:
    poisson_dir = f"./smooth_experiments/poisson/{i}/"
    poisson_tri = poisson_dir + f"{i}poisson.off"

    poisson_output_dir = poisson_dir + "results/"
    poisson_lap_dir = poisson_output_dir + "lap/"
    poisson_con_lap_dir = poisson_output_dir + "con_lap/"
    poisson_distmesh_dir = poisson_output_dir + "distmesh/"
    Path(poisson_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(poisson_con_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(poisson_distmesh_dir).mkdir(parents=True, exist_ok=True)

    analytics = poisson_output_dir + "analytics.csv"
    create_analytics_file(analytics)
    
    subprocess.run(["python.exe", kernel_exec, poisson_output_dir + "poisson_vanilla.off"], stdout=subprocess.DEVNULL)
    subprocess.run(["python.exe", analytics_exec, poisson_output_dir + "poisson_vanilla.off", analytics], stdout=subprocess.DEVNULL)

    # laplacian
    print("laplacian")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, poisson_lap_dir + f"poisson_{n}lap-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, poisson_lap_dir + f"poisson_{n}lap-0.off", analytics], stdout=subprocess.DEVNULL)
        
    # constrained laplacian
    print("constrained laplacian")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, poisson_con_lap_dir + f"poisson_{n}con_lap-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, poisson_con_lap_dir + f"poisson_{n}con_lap-0.off", analytics], stdout=subprocess.DEVNULL)
        
    # distmesh
    print("distmesh")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, poisson_distmesh_dir + f"poisson_{n}distmesh-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, poisson_distmesh_dir + f"poisson_{n}distmesh-0.off", analytics], stdout=subprocess.DEVNULL)

laplacian
n: 10
n: 20
constrained laplacian
n: 10
n: 20
distmesh
n: 10
n: 20
laplacian
n: 10
n: 20
constrained laplacian
n: 10
n: 20
distmesh
n: 10
n: 20
laplacian
n: 10
n: 20
constrained laplacian
n: 10
n: 20
distmesh
n: 10
n: 20


In [18]:
for p in points:
    poisson_dir = f"./smooth_experiments/poisson/{p}/"

    poisson_output_dir = poisson_dir + "results/"
    poisson_lap_dir = poisson_output_dir + "lap/"
    poisson_con_lap_dir = poisson_output_dir + "con_lap/"
    poisson_distmesh_dir = poisson_output_dir + "distmesh/"

    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(poisson_lap_dir + f"poisson_20lap-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/n
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]
    print("Laplacian", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))
    print()

    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(poisson_con_lap_dir + f"poisson_20con_lap-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/20
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

    print("Constrained Laplacian", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))
    print()

    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(poisson_distmesh_dir + f"poisson_20distmesh-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/n
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

    print("Distmesh", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))
    print()

Laplacian 100
Total: 0.8424700000000002
Smooth 0.5893499999999999
Avg: 0.0294675
Remaining: 0.25312
One iter: 0.10427743619232982

Constrained Laplacian 100
Total: 19.06554
Smooth 18.812060000000002
Avg: 0.940603
Remaining: 0.2534799999999997
One iter: 0.7877199491157652

Distmesh 100
Total: 25.04264
Smooth 24.76432
Avg: 1.238216
Remaining: 0.27832000000000046
One iter: 0.8164764964366159

Laplacian 1000
Total: 10.50724
Smooth 7.776599999999999
Avg: 0.38883
Remaining: 2.7306399999999997
One iter: 0.12464617386927909

Constrained Laplacian 1000
Total: 235.78589999999994
Smooth 233.1932
Avg: 11.659660000000002
Remaining: 2.5926999999999962
One iter: 0.8180862678181019

Distmesh 1000
Total: 295.0497
Smooth 292.55850000000004
Avg: 14.627925000000001
Remaining: 2.491199999999992
One iter: 0.8544785437339821

Laplacian 10000
Total: 105.1482
Smooth 79.34682000000001
Avg: 3.967341
Remaining: 25.80138
One iter: 0.13327213486934825

Constrained Laplacian 10000
Total: 2447.7780000000002
Smooth 24

In [11]:

for i in points:
    random_dir = f"./smooth_experiments/random/{i}/"
    random_tri = random_dir + f"{i}random.off"

    random_output_dir = random_dir + "results/"
    random_lap_dir = random_output_dir + "lap/"
    random_con_lap_dir = random_output_dir + "con_lap/"
    random_distmesh_dir = random_output_dir + "distmesh/"
    Path(random_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(random_con_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(random_distmesh_dir).mkdir(parents=True, exist_ok=True)

    subprocess.run([polylla_exec, random_tri, random_output_dir + "random_vanilla"], stdout=subprocess.DEVNULL)

    # laplacian
    print("laplacian")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, random_tri, random_lap_dir + f"random_{n}lap-{i}", "--smooth", "laplacian", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

    # constrained laplacian
    print("constrained laplacian")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, random_tri, random_con_lap_dir + f"random_{n}con_lap-{i}", "--smooth", "laplacian-edge-ratio", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

    # distmesh
    print("distmesh")
    for n in iter:
        print("n:",n)
        for i in range(repeat):
            print("i:",i,end=" ")
            subprocess.run([polylla_exec, random_tri, random_distmesh_dir + f"random_{n}distmesh-{i}", "--smooth", "distmesh", "--smooth-iter", str(n)], stdout=subprocess.DEVNULL)
        print("")

laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
constrained laplacian
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 
distmesh
n: 20
i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 


In [80]:
for i in points:
    random_dir = f"./smooth_experiments/random/{i}/"
    random_tri = random_dir + f"{i}random.off"

    random_output_dir = random_dir + "results/"
    random_lap_dir = random_output_dir + "lap/"
    random_con_lap_dir = random_output_dir + "con_lap/"
    random_distmesh_dir = random_output_dir + "distmesh/"
    Path(random_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(random_con_lap_dir).mkdir(parents=True, exist_ok=True)
    Path(random_distmesh_dir).mkdir(parents=True, exist_ok=True)

    analytics_random = random_output_dir + "analytics.csv"
    create_analytics_file(analytics_random)
    
    subprocess.run(["python.exe", kernel_exec, random_output_dir + "random_vanilla.off"], stdout=subprocess.DEVNULL)
    subprocess.run(["python.exe", analytics_exec, random_output_dir + "random_vanilla.off", analytics_random], stdout=subprocess.DEVNULL)

    # laplacian
    print("laplacian")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, random_lap_dir + f"random_{n}lap-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, random_lap_dir + f"random_{n}lap-0.off", analytics_random], stdout=subprocess.DEVNULL)

    # constrained laplacian
    print("constrained laplacian")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, random_con_lap_dir + f"random_{n}con_lap-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, random_con_lap_dir + f"random_{n}con_lap-0.off", analytics_random], stdout=subprocess.DEVNULL)
        
    # distmesh
    print("distmesh")
    for n in iter:
        print("n:",n)
        subprocess.run(["python.exe", kernel_exec, random_distmesh_dir + f"random_{n}distmesh-0.off"], stdout=subprocess.DEVNULL)
        subprocess.run(["python.exe", analytics_exec, random_distmesh_dir + f"random_{n}distmesh-0.off", analytics_random], stdout=subprocess.DEVNULL)



laplacian
n: 10
n: 20
constrained laplacian
n: 10
n: 20
distmesh
n: 10
n: 20
laplacian
n: 10
n: 20
constrained laplacian
n: 10
n: 20
distmesh
n: 10
n: 20
laplacian
n: 10
n: 20
constrained laplacian
n: 10
n: 20
distmesh
n: 10
n: 20


In [19]:
for p in points:
    random_dir = f"./smooth_experiments/random/{p}/"

    random_output_dir = random_dir + "results/"
    random_lap_dir = random_output_dir + "lap/"
    random_con_lap_dir = random_output_dir + "con_lap/"
    random_distmesh_dir = random_output_dir + "distmesh/"

    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(random_lap_dir + f"random_20lap-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/n
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]
    print("Laplacian", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))
    print()

    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(random_con_lap_dir + f"random_20con_lap-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/20
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

    print("Constrained Laplacian", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))
    print()

    total_time = 0
    smooth_time = 0
    avg_smooth_time = 0
    remaining_time = 0
    for i in range(repeat):
        with open(random_distmesh_dir + f"random_20distmesh-{i}.json") as f:
            data = json.load(f)
            n = data["n_smooth_iterations"]
            smooth_time += data["time_to_smooth"]
            avg_smooth_time += data["time_to_smooth"]/n
            total_time += data["time_to_generate_polygonal_mesh"]
            remaining_time += data["time_to_generate_polygonal_mesh"] - data["time_to_smooth"]

    print("Distmesh", p)
    print("Total:",total_time/repeat)
    print("Smooth", smooth_time/repeat)
    print("Avg:", avg_smooth_time/repeat)
    print("Remaining:", remaining_time/repeat)
    print("One iter:", avg_smooth_time/(remaining_time + avg_smooth_time))
    print()

Laplacian 100
Total: 0.8337
Smooth 0.56493
Avg: 0.028246499999999997
Remaining: 0.26876999999999995
One iter: 0.09510077722954785

Constrained Laplacian 100
Total: 39.37012
Smooth 39.098850000000006
Avg: 1.9549425
Remaining: 0.2712700000000005
One iter: 0.8781473017512926

Distmesh 100
Total: 38.41749
Smooth 38.15886
Avg: 1.907943
Remaining: 0.25863000000000225
One iter: 0.8806271471120511

Laplacian 1000
Total: 10.440600000000002
Smooth 7.3812099999999985
Avg: 0.36906049999999996
Remaining: 3.0593899999999996
One iter: 0.10764644261306967

Constrained Laplacian 1000
Total: 471.9026
Smooth 469.35450000000003
Avg: 23.467725
Remaining: 2.548100000000011
One iter: 0.9020557679796812

Distmesh 1000
Total: 481.6786
Smooth 479.20520000000005
Avg: 23.960259999999998
Remaining: 2.473399999999998
One iter: 0.9064299079279979

Laplacian 10000
Total: 98.34112
Smooth 71.80938
Avg: 3.590469
Remaining: 26.531740000000003
One iter: 0.11919673620218224

Constrained Laplacian 10000
Total: 5090.283
Smoo